In [1]:
import os, sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
os.environ['PYTHONPATH'] = project_root
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("PYTHONPATH manually set to:", os.environ['PYTHONPATH'])

PYTHONPATH manually set to: c:\Users\ndhaj\Desktop\GraphicalPR


In [2]:
#Gaussian Prior
import numpy as np
from graph.structure.graph import Graph
from graph.prior.gaussian_prior import GaussianPrior

rng = np.random.default_rng(seed=42)
X = GaussianPrior(shape = (512,))
X.parent.generate_sample(rng)
x = X.get_sample()
print(f"Mean: {np.mean(x):.3f}, Variance: {np.var(x):.3f}")

Mean: -0.013-0.029j, Variance: 0.967


In [3]:
import numpy as np
from graph.prior.sparse_prior import SparsePrior

# パラメータ設定
rho = 0.1
shape = (100,)
expected_nonzero = int(rho * np.prod(shape))

# Priorインスタンスとランダム生成
X = SparsePrior(rho=rho, shape=shape)
rng = np.random.default_rng(seed=123)
X.parent.generate_sample(rng)

# サンプル取得とチェック
x = X.get_sample()

print("Sample:", x[:10])  # 最初の10個だけ表示
print("Nonzero count:", np.count_nonzero(x))
print("Expected approx:", expected_nonzero)

# 簡易アサーション（許容誤差 ±2）
assert x.shape == shape
assert x.dtype == np.complex128
assert expected_nonzero - 2 <= np.count_nonzero(x) <= expected_nonzero + 2

Sample: [0.        +0.j         0.06870767-1.56091778j 0.        +0.j
 0.        +0.j         0.        +0.j         0.23883884+0.00361637j
 0.        +0.j         0.        +0.j         0.        +0.j
 0.        +0.j        ]
Nonzero count: 10
Expected approx: 10


In [4]:
from graph.propagator.unitary_propagator import UnitaryPropagator
from graph.prior.sparse_prior import SparsePrior

# 入力をスパースベクトルに
X = SparsePrior(rho=0.2, shape=(8,))
U = np.eye(8, dtype=np.complex128)  # identity でも良い（単位行列）

P = UnitaryPropagator(U)
Y = P @ X

rng = np.random.default_rng(seed=123)
X.parent.generate_sample(rng)
P.generate_sample(rng)

x_sample = X.get_sample()
y_sample = Y.get_sample()

print("x:", x_sample)
print("y:", y_sample)
print("U @ x:", U @ x_sample)
assert np.allclose(y_sample, U @ x_sample)

x: [-0.26006444+0.91070069j  0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j        ]
y: [-0.26006444+0.91070069j  0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j        ]
U @ x: [-0.26006444+0.91070069j  0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j        ]


In [5]:
import numpy as np
from graph.prior.sparse_prior import SparsePrior
from graph.measurement.gaussian_measurement import GaussianMeasurement

# 1. グラフ構築
X = SparsePrior(rho=0.2, shape=(8,))
Z = GaussianMeasurement(X, var = 1e-4)

# 2. サンプル生成
rng = np.random.default_rng(seed=42)
X.parent.generate_sample(rng)
Z.generate_sample(rng)

# 3. 観測値として反映
Z.update_observed_from_sample()

# 4. 確認
x_true = X.get_sample()
y_noisy = Z.get_sample()
observed = Z.observed.data

print("x (true signal):", x_true)
print("y (noisy):", y_noisy)
print("observed (for inference):", observed)

assert np.allclose(y_noisy, observed)


x (true signal): [-0.73537981+0.53064913j  0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j          0.        +0.j
  0.        +0.j          0.        +0.j        ]
y (noisy): [-7.28729017e-01+5.36148949e-01j -1.37959021e-02+4.66907540e-04j
 -9.20779960e-03+7.97079901e-03j  9.03968160e-04+3.30579026e-03j
 -2.23617282e-03-6.07611528e-03j -1.18802124e-04+2.60746180e-03j
 -6.03193146e-03-6.78032389e-03j  6.21828271e-03+6.21158165e-03j]
observed (for inference): [-7.28729017e-01+5.36148949e-01j -1.37959021e-02+4.66907540e-04j
 -9.20779960e-03+7.97079901e-03j  9.03968160e-04+3.30579026e-03j
 -2.23617282e-03-6.07611528e-03j -1.18802124e-04+2.60746180e-03j
 -6.03193146e-03-6.78032389e-03j  6.21828271e-03+6.21158165e-03j]


In [6]:
import numpy as np
from graph.prior.sparse_prior import SparsePrior
from graph.measurement.masked_gaussian_measurement import MaskedGaussianMeasurement
from core.linalg_utils import random_binary_mask

# --- パラメータ設定 ---
n = 8
rho = 0.25
var = 1e-2
mask = random_binary_mask(n, subsampling_rate=0.5, seed=0)

# --- グラフ構築（観測データなしで初期化） ---
X = SparsePrior(rho=rho, shape=(n,))
Z = MaskedGaussianMeasurement(X, var=var, mask=mask)  # observed_data=None

# --- サンプル生成 ---
rng = np.random.default_rng(seed=123)
X.parent.generate_sample(rng)
Z.generate_sample(rng)
Z.update_observed_from_sample()

# --- 結果確認 ---
x_true = X.get_sample()
y_noisy = Z.get_sample()
observed = Z.observed.data

print("True signal x:", x_true)
print("Noisy observation y:", y_noisy)
print("Observed (used in inference):", observed)
print("Mask:", mask.astype(int))

# --- 整合性チェック ---
assert y_noisy.shape == (n,)
assert np.allclose(y_noisy, observed)
assert np.all(Z.observed._precision == np.where(mask, 1.0 / var, 0.0))


True signal x: [0.91070069+0.65070151j 0.        +0.j         0.        +0.j
 0.        +0.j         0.        +0.j         0.13716063+0.408074j
 0.        +0.j         0.        +0.j        ]
Noisy observation y: [ 0.86569591+0.65757228j  0.        +0.j          0.03832181-0.10789957j
  0.        +0.j          0.        +0.j          0.11477395+0.49237288j
 -0.02279635-0.04745321j  0.        +0.j        ]
Observed (used in inference): [ 0.86569591+0.65757228j  0.        +0.j          0.03832181-0.10789957j
  0.        +0.j          0.        +0.j          0.11477395+0.49237288j
 -0.02279635-0.04745321j  0.        +0.j        ]
Mask: [1 0 1 0 0 1 1 0]
